<a href="https://colab.research.google.com/github/daryoush/minGPT/blob/master/startjulia1_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [9]:
%cd /content/gdrive/My Drive/
#! git clone https://github.com/daryoush/minGPT.git
%cd minGPT
! git pull
#! pip install <desired-python-library>

The equivalent of `%cd 'dir'` in IPython is `cd("dir")` in Julia.


The equivalent of `%cd 'dir'` in IPython is `cd("dir")` in Julia.


# New Section

In [4]:
### NOTE THIS WORKS AS PYTHON, IT ONLY WORKS WHEN THE RUNTIME COMPLAINS ABOUT JULIA RUNTIME, SO IT REVERTS TO PYTHON. AFTER JULIA IS INSTALLED, CHANGE TO JULIA RUNTIME

!curl -sSL "https://julialang-s3.julialang.org/bin/linux/x64/1.5/julia-1.5.0-linux-x86_64.tar.gz" -o julia.tar.gz
!tar -xzf julia.tar.gz -C /usr --strip-components 1
!rm -rf julia.tar.gz*
!julia -e 'using Pkg; pkg"add IJulia; precompile"'
!julia -e 'using Pkg; Pkg.add([ "CUDA", "Flux","Zygote","Profile", "TensorBoardLogger", "Logging",\
    "Random","InformationMeasures", "DataStructures", "Transformers","Statistics", "Dates",\
    "BSON",\
    "Images",\
    "MLDatasets",\
    "Parameters",\
    "Printf",\
    ]);' ###Pkg.precompile()'
!echo "DONE"

 Installing known registries into `~/.julia`
######################################################################## 100.0%
      Added registry `General` to `~/.julia/registries/General`
  Resolving package versions...
  Installed Conda ─────────── v1.5.0
  Installed ZeroMQ_jll ────── v4.3.2+5
  Installed VersionParsing ── v1.2.0
  Installed SoftGlobalScope ─ v1.1.0
  Installed IJulia ────────── v1.23.1
  Installed ZMQ ───────────── v1.2.1
  Installed JSON ──────────── v0.21.1
  Installed JLLWrappers ───── v1.1.3
  Installed Artifacts ─────── v1.3.0
  Installed MbedTLS_jll ───── v2.16.8+1
  Installed MbedTLS ───────── v1.0.3
  Installed Parsers ───────── v1.0.15
######################################################################## 100.0%
######################################################################## 100.0%
Updating `~/.julia/environments/v1.5/Project.toml`
  [7073ff75] + IJulia v1.23.1
Updating `~/.julia/environments/v1.5/Manifest.toml`
  [56f22d72] + Artifacts v1.3.0
  

**NOW GO TO EDIT NOTEBOOK SETTING AND SET IT TO JULIA**
For adding new packages, it should be in python or the syntax may need to be in Julia!

In [1]:
[1,2,3]

3-element Array{Int64,1}:
 1
 2
 3

In [2]:
import Pkg; Pkg.add("BenchmarkTools")

   Updating registry at `~/.julia/registries/General`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`


In [12]:
using BenchmarkTools

M = rand(2048, 2048)
@benchmark M^2

BenchmarkTools.Trial: 
  memory estimate:  32.00 MiB
  allocs estimate:  2
  --------------
  minimum time:     417.434 ms (0.00% GC)
  median time:      431.325 ms (0.00% GC)
  mean time:        435.868 ms (0.73% GC)
  maximum time:     471.040 ms (0.71% GC)
  --------------
  samples:          12
  evals/sample:     1

In [13]:
if ENV["COLAB_GPU"] == "1"
    using CUDA

    M_gpu = cu(M)
    @benchmark CUDA.@sync M_gpu^2
else
    println("No GPU found.")
end

BenchmarkTools.Trial: 
  memory estimate:  416 bytes
  allocs estimate:  11
  --------------
  minimum time:     2.757 ms (0.00% GC)
  median time:      4.951 ms (0.00% GC)
  mean time:        5.025 ms (0.03% GC)
  maximum time:     9.150 ms (17.12% GC)
  --------------
  samples:          994
  evals/sample:     1

In [17]:
M_gpu = nothing
M=nothing

In [2]:
cd("/content/gdrive/MyDrive/minGPT")
pwd()

"/content/gdrive/MyDrive/minGPT"

In [3]:
using Flux: onehot
	using Transformers: Vocabulary, decode
	using Random
	using Flux: throttle, @epochs, ADAMW, ADAM, RADAM, Momentum, Optimiser, logitcrossentropy, params
	using Flux: softmax, onecold
	using DataStructures: Queue, enqueue!, dequeue!
	Random.seed!(42)


MersenneTwister(UInt32[0x0000002a], Random.DSFMT.DSFMT_state(Int32[964434469, 1073036706, 1860149520, 1073503458, 1687169063, 1073083486, -399267803, 1072983952, -909620556, 1072836235  …  -293054293, 1073002412, -1300127419, 1073642642, 1917177374, -666058738, -337596527, 1830741494, 382, 0]), [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], UInt128[0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000  …  0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x0000000000

In [4]:
import Pkg; Pkg.add("NNlib")
Pkg.add("Distributions")
Pkg.add("Juno")

   Updating registry at `~/.julia/registries/General`
######################################################################### 100.0%
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`


In [5]:
using Flux

module Datasets include("onJulia/src/datasets.jl") end

Main.Datasets

In [20]:
f="input.txt"
s = open(f) do file
      read(file, String)
end
data = s |> gpu
xxx=10    # I don't want the out put of the data assignment to be printed

10

In [7]:
using CUDA
CUDA.memory_status()

Effective GPU memory usage: 0.73% (109.875 MiB/14.726 GiB)
CUDA allocator usage: 0 bytes
binned usage: 0 bytes (0 bytes allocated, 0 bytes cached)


In [8]:

dataSetlength, vocab, sample = Datasets.asDataset(data)
batch(dataLength,batchLength ) = hcat((sample(dataLength) for x in 1:batchLength)...)

datasize = length(s) = 1115394


batch (generic function with 1 method)

In [9]:
## ---
module Model include("onJulia/src/model.jl") end

lengthOfSentences=520

520

In [10]:
model=Model.minGPT(numberAttnLayers=4, maxBlockSize=lengthOfSentences, vocabSize=length(vocab)) |> gpu

Main.Model.minGPT{Transformers.Basic.Embed{Float32,CuArray{Float32,2}},Main.Model.GPTPositionEmbedding{Float32,CuArray{Float32,2}},Chain{NTuple{4,Transformers.Basic.Transformer{Transformers.Basic.MultiheadAttention{Dense{typeof(identity),CuArray{Float32,2},CuArray{Float32,1}},Dense{typeof(identity),CuArray{Float32,2},CuArray{Float32,1}},Dense{typeof(identity),CuArray{Float32,2},CuArray{Float32,1}},Dense{typeof(identity),CuArray{Float32,2},CuArray{Float32,1}},Dropout{Float64,Colon}},LayerNorm{CuArray{Float32,1}},Transformers.Basic.PwFFN{Dense{typeof(gelu),CuArray{Float32,2},CuArray{Float32,1}},Dense{typeof(identity),CuArray{Float32,2},CuArray{Float32,1}}},LayerNorm{CuArray{Float32,1}},Dropout{Float64,Colon}}}},Dropout{Float64,Colon},LayerNorm{CuArray{Float32,1}},Transformers.Basic.Positionwise{Tuple{Dense{typeof(identity),CuArray{Float32,2},CuArray{Float32,1}}}}}(Embed(512), Main.Model.GPTPositionEmbedding{Float32,CuArray{Float32,2}}(Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0

In [11]:
@show modelParmaLength=sum(length(p) for p in params(model))
CUDA.memory_status()

modelParmaLength = sum((length(p) for p = params(model))) = 12944450
Effective GPU memory usage: 1.07% (161.875 MiB/14.726 GiB)
CUDA allocator usage: 50.606 MiB
binned usage: 50.606 MiB (50.606 MiB allocated, 0 bytes cached)


In [12]:
# for a batch of sentences, extract 1st to one before last character as input
# and 2nd to last char of each sentence as prediction.  Shape the predictred
# values as a long one dim series of values, one hot encode them using the vocab

trainingDataPair(b)=(b[1:end-1, :], onehot(vocab, reshape(b[2:end,:], :)))
trainingData(dataLength,batchLength)= [trainingDataPair(batch(dataLength,batchLength))
		for i in 1:dataSetlength/(dataLength*batchLength)]

function loss(x,y)
	yhat=model(x)
	logitcrossentropy(reshape(yhat,size(yhat)[1], :), y)
end


function validate()
	s=collect("Wher")
	for i in 1:20
		out=model(vocab(s))
		push!(s, decode(vocab,onecold(softmax(out)[:,end])))
	end
	join(s)
end

validate (generic function with 1 method)

In [13]:
opt = RADAM()
#opt = ADAM( 0.0006, (0.9, 0.95))

## ---
module Train include("onJulia/src/train.jl") end

Main.Train

In [14]:
function sampleFromModel(s="Whe", minprob=.05) 
	function explore(x, minprob)
		x[1] < minprob && return currentRes
		out=model(vocab(collect(x[2])))
		choices=sort(collect(zip(softmax(out[:,end]), decode(vocab, collect(1:size(out)[1])))), rev=true)
		options = [(x[1]*np, x[2]*nc) for (np,nc) in choices if (x[1]*np) > minprob ]
	end
	toexplore=Queue{Tuple{Float32,String}}()
	enqueue!(toexplore,(1, s))  #start with the initial string and prob 1
	results=[]
	for i in toexplore
		nxt=dequeue!(toexplore)
		res=explore(nxt, minprob)
		if isempty(res)  # go as long as you can't add any more character and satisfy the prob
			push!(results, nxt)
			continue #continue emptying the queue
		end
		for r in res
			enqueue!(toexplore, r)  ## add elements for further exploration
		end
	end
    #elements of the results are (prob, string), sort from high to low of the 
	#longest match with highest prob
	sort(results, by=x->(length(x[2]), x[1]), rev=true)
end


sampleFromModel (generic function with 3 methods)

In [15]:
sampleFromModel("Where i", 0.01)

┌ Warning: Performing scalar operations on GPU arrays: This is very slow, consider disallowing these operations with `allowscalar(false)`
└ @ GPUArrays /root/.julia/packages/GPUArrays/uaFZh/src/host/indexing.jl:43


50-element Array{Any,1}:
 (0.044868156f0, "Where iq")
 (0.04141272f0, "Where iC")
 (0.040363334f0, "Where iJ")
 (0.03673053f0, "Where iO")
 (0.032043885f0, "Where ih")
 (0.02669385f0, "Where ig")
 (0.026286285f0, "Where ir")
 (0.024210459f0, "Where i□")
 (0.023948336f0, "Where iU")
 (0.02156169f0, "Where i:")
 (0.020549884f0, "Where i?")
 (0.020101245f0, "Where iF")
 (0.020039162f0, "Where ij")
 ⋮
 (0.011340568f0, "Where im")
 (0.011208045f0, "Where ix")
 (0.011195152f0, "Where iB")
 (0.01110347f0, "Where if")
 (0.010874684f0, "Where i\n")
 (0.01087423f0, "Where iA")
 (0.010867239f0, "Where ib")
 (0.010675254f0, "Where iR")
 (0.01058289f0, "Where i'")
 (0.010293674f0, "Where iS")
 (0.010070103f0, "Where iP")
 (0.010061818f0, "Where iE")

In [16]:
numberOfSentencesInBatch=10
ps = params(model)
for i in 1:50 
  show "epoch: ", i
  d=trainingData(lengthOfSentences, numberOfSentencesInBatch) |> gpu
  Train.train!(loss,
            ps,
            d,
            opt,
            sampleFromModel )
end



Streaming output truncated to the last 5000 lines.
ctr = 132
ctr = 133
ctr = 134
ctr = 135
ctr = 136
ctr = 137
ctr = 138
ctr = 139
ctr = 140
(ctr, training_loss) = (140, 1.2092271f0)
training_accuracy = validate() = Any[(0.27766943f0, "Where "), (0.07180346f0, "When w"), (0.061453193f0, "When h")]
ctr = 141
ctr = 142
ctr = 143
ctr = 144
ctr = 145
ctr = 146
ctr = 147
ctr = 148
ctr = 149
ctr = 150
(ctr, training_loss) = (150, 1.1470917f0)
training_accuracy = validate() = Any[(0.060544774f0, "Where i"), (0.053393815f0, "When h")]
ctr = 151
ctr = 152
ctr = 153
ctr = 154
ctr = 155
ctr = 156
ctr = 157
ctr = 158
ctr = 159
ctr = 160
(ctr, training_loss) = (160, 1.1845375f0)
training_accuracy = validate() = Any[(0.08808118f0, "Where is "), (0.05249132f0, "When you"), (0.06437191f0, "When w")]
ctr = 161
ctr = 162
ctr = 163
ctr = 164
ctr = 165
ctr = 166
ctr = 167
ctr = 168
ctr = 169
ctr = 170
(ctr, training_loss) = (170, 1.1928697f0)
training_accuracy = validate() = Any[(0.0746745f0, "When you ")

In [14]:
Pkg.add("BSON")
using BSON

  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`


In [15]:
using BSON: 
@save "inputtext_model.bson"  model

In [16]:
using Zygote
using Flux
using Transformers
using CUDA
using BSON:  @save, @load
@load "inputtext_model.bson"  model


In [19]:
sampleFromModel("Where is King Richard Duke of York, and ", 0.001)


LoadError: ignored

In [20]:
Pkg.add("StatsBase")
Pkg.add("Plots")

  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
  Installed Ogg_jll ───────── v1.3.4+2
  Installed LAME_jll ──────── v3.100.0+3
  Installed x265_jll ──────── v3.0.0+3
  Installed PlotThemes ────── v1.0.3
  Installed RecipesPipeline ─ v0.1.13
  Installed Plots ─────────── v1.0.14
  Installed Contour ───────── v0.5.7
  Installed GeometryTypes ─── v0.8.4
  Installed Showoff ───────── v0.3.2
  Installed Grisu ─────────── v1.0.0
  Installed FreeType2_jll ─── v2.10.1+5
  Installed Opus_jll ──────── v1.3.1+3
  Installed FFMPEG_jll ────── v4.3.1+4
  Installed LibVPX_jll ────── v1.9.0+1
  Installed libfdk_aac_jll ── v0.1.6+4
  Installed PlotUtils ─────── v0.6.5
  Installed libvorbis_jll ─── v1.3.6+6
  Installed FFMPEG ────────── v0.3.0
  Installed Bzip2_jll ─────── v1.0.6+5
  Installed Measures ──────── v0.3.1
  Installed FriBidi_jll ───── v1.0.5+6
  Installed libass_

In [47]:
using StatsBase
using Plots
function getOutputResults(m, data, limit=5)
	output=[]
	ctr=0
	for (x,y) in data
		out=softmax(m(x))
		out=reshape(out,size(out)[1], :)
		push!(output, onecold(out)...)  ## get the encoded vocab using straight onecold, not onecold(vocab, ..) version

		ctr +=1
		if  ctr > limit break end
	end
	convert(Array{Int}, output)
end

d=trainingData(lengthOfSentences, numberOfSentencesInBatch)
function OutputHistogram(m)
	outChars=getOutputResults(m, d)
	fit(Histogram, outChars, 0:1:length(vocab))
end

#Try a untrained model
raw=OutputHistogram(Model.minGPT(maxBlockSize=lengthOfSentences, vocabSize=length(vocab)))

#..a. and trained model
trained=OutputHistogram(model)

plot(raw)
plot!(trained)

LoadError: ignored

In [75]:
data = s |> gpu
collect(data)[1:10]

10-element Array{Char,1}:
 'F': ASCII/Unicode U+0046 (category Lu: Letter, uppercase)
 'i': ASCII/Unicode U+0069 (category Ll: Letter, lowercase)
 'r': ASCII/Unicode U+0072 (category Ll: Letter, lowercase)
 's': ASCII/Unicode U+0073 (category Ll: Letter, lowercase)
 't': ASCII/Unicode U+0074 (category Ll: Letter, lowercase)
 ' ': ASCII/Unicode U+0020 (category Zs: Separator, space)
 'C': ASCII/Unicode U+0043 (category Lu: Letter, uppercase)
 'i': ASCII/Unicode U+0069 (category Ll: Letter, lowercase)
 't': ASCII/Unicode U+0074 (category Ll: Letter, lowercase)
 'i': ASCII/Unicode U+0069 (category Ll: Letter, lowercase)

In [71]:
GC.gc(true)
CUDA.memory_status()


Effective GPU memory usage: 99.97% (14.723 GiB/14.726 GiB)
CUDA allocator usage: 14.540 GiB
binned usage: 14.540 GiB (115.742 MiB allocated, 14.427 GiB cached)


In [19]:
model=nothing; GC.gc(true)

In [20]:
CUDA.memory_status()


Effective GPU memory usage: 8.28% (1.219 GiB/14.726 GiB)
CUDA allocator usage: 1.047 GiB
binned usage: 1.047 GiB (0 bytes allocated, 1.047 GiB cached)


In [21]:
CUDA.reclaim()

1124073472

In [22]:
CUDA.memory_status()


Effective GPU memory usage: 1.17% (175.875 MiB/14.726 GiB)
CUDA allocator usage: 0 bytes
binned usage: 0 bytes (0 bytes allocated, 0 bytes cached)
